In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import json
import csv
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.util import bigrams
from google.colab import files
uploaded = files.upload()

In [ ]:
names = ["web1.json", "web-3.json", "web-4.json",
         "web-1005.json", "web-2012.json", 
         "web-10034.json", "web-10047.json", 
         "web-10089.json", "web-10133.json", 
         "web-10165.json"]
data_list=[]
for i in names:
  data_list.append(uploaded[i].decode("utf-8"))
data_list

In [5]:
df_data1 = pd.DataFrame(zip(data_list, data_list),columns =['Raw_Text', 'Processed_Text'])
df_data1.head()

,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...","{""text"": [""2021 Coachmen Sportscoach 402TS Two..."
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...","{""text"": [""Jemarl Baker Jr. to transfer from A..."
2,"{""text"": [""Publications | The Protect Heritage...","{""text"": [""Publications | The Protect Heritage..."
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...","{""text"": ["" Arts vs. Athletics: Two Great Ways..."
4,"{\r\n ""title"": [\r\n ""Purposeful Inv...","{\r\n ""title"": [\r\n ""Purposeful Inv..."


#(1)

In [6]:
for i in range(10):
  df_data1['Processed_Text'].values[i] = (re.sub(r"\W", " ", df_data1['Processed_Text'].values[i]))
  df_data1['Processed_Text'].values[i] = (re.sub(r" n ", " ", df_data1['Processed_Text'].values[i]))
  df_data1['Processed_Text'].values[i] = (re.sub(r" t ", " ", df_data1['Processed_Text'].values[i]))
  df_data1['Processed_Text'].values[i] = (re.sub(r" r ", " ", df_data1['Processed_Text'].values[i]))
  df_data1['Processed_Text'].values[i] = " ".join(df_data1['Processed_Text'].values[i].split())
  df_data1['Processed_Text'].values[i] = df_data1['Processed_Text'].values[i].split(' ', 1)[1]
df_data1.head()

,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...",2021 Coachmen Sportscoach 402TS Two Full Bath ...
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...",Jemarl Baker Jr to transfer from Arizona Wildc...
2,"{""text"": [""Publications | The Protect Heritage...",Publications The Protect Heritage Corp Home Ab...
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...",Arts vs Athletics Two Great Ways You Can Prepa...
4,"{\r\n ""title"": [\r\n ""Purposeful Inv...",Purposeful Investing Legacy Bridge Private Fam...


In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop = stopwords.words('english')
stop.append('n')
stop.append('r')
lem = WordNetLemmatizer()
stem = PorterStemmer()
for i in range(10):
   x = df_data1['Processed_Text'].values[i]
   token = nltk.word_tokenize(x)
   df_data1['Processed_Text'].values[i] = token
   c = [t for t in token if not t in stop]
   df_data1['Processed_Text'].values[i] = c
df_data1.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...","[2021, Coachmen, Sportscoach, 402TS, Two, Full..."
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...","[Jemarl, Baker, Jr, transfer, Arizona, Wildcat..."
2,"{""text"": [""Publications | The Protect Heritage...","[Publications, The, Protect, Heritage, Corp, H..."
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...","[Arts, vs, Athletics, Two, Great, Ways, You, C..."
4,"{\r\n ""title"": [\r\n ""Purposeful Inv...","[Purposeful, Investing, Legacy, Bridge, Privat..."


In [8]:
for x in range(10):
  z = []
  y = df_data1['Processed_Text'].values[x]
  for j in y:
    z.append(stem.stem(j))
  df_data1['Processed_Text'].values[x] = z
df_data1.head()

,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...","[2021, coachmen, sportscoach, 402t, two, full,..."
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...","[jemarl, baker, Jr, transfer, arizona, wildcat..."
2,"{""text"": [""Publications | The Protect Heritage...","[public, the, protect, heritag, corp, home, ab..."
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...","[art, vs, athlet, two, great, way, you, can, p..."
4,"{\r\n ""title"": [\r\n ""Purposeful Inv...","[purpos, invest, legaci, bridg, privat, famili..."


In [9]:
nltk.download('wordnet')
for x in range(10):
  z = []
  y = df_data1['Processed_Text'].values[x]
  for j in y:
    z.append(lem.lemmatize(j))
  df_data1['Processed_Text'].values[x] = z
df_data1.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...","[2021, coachman, sportscoach, 402t, two, full,..."
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...","[jemarl, baker, Jr, transfer, arizona, wildcat..."
2,"{""text"": [""Publications | The Protect Heritage...","[public, the, protect, heritag, corp, home, ab..."
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...","[art, v, athlet, two, great, way, you, can, pr..."
4,"{\r\n ""title"": [\r\n ""Purposeful Inv...","[purpos, invest, legaci, bridg, privat, famili..."


# Term Frequency Inverse Document Frequency (TF-IDF)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
for i in range(10):
  y = df_data1['Processed_Text'].values[i]
  print("\nIDF values of Text Document no.", i)
  result = tfidf.fit_transform(y)
  for a, b in zip(tfidf.get_feature_names(), tfidf.idf_):
    print(a, ':', b)
  print('\nWord Indexing')
  print(tfidf.vocabulary_)
  print("\nTF-IDF values of Text document no.", i)
  print(result)



IDF values of Text Document no. 0
000 : 6.625820933186489
001 : 6.625820933186489
00a : 6.625820933186489
00p : 6.625820933186489
10 : 6.220355825078324
100 : 6.220355825078324
15yr : 6.625820933186489
20 : 6.625820933186489
2019 : 6.625820933186489
2020 : 6.625820933186489
2021 : 6.220355825078324
20th : 6.220355825078324
25 : 6.625820933186489
319 : 6.625820933186489
335 : 5.932673752626544
35w : 6.625820933186489
369 : 6.625820933186489
40 : 6.625820933186489
402t : 6.220355825078324
42 : 6.625820933186489
45 : 6.625820933186489
46 : 6.625820933186489
49 : 6.220355825078324
50k : 6.625820933186489
5411 : 6.625820933186489
555 : 6.625820933186489
6054 : 5.932673752626544
6395 : 6.625820933186489
685 : 6.625820933186489
76009 : 6.220355825078324
7771 : 5.932673752626544
783 : 6.625820933186489
790 : 5.932673752626544
800 : 5.932673752626544
817 : 5.709530201312334
913 : 6.625820933186489
949 : 6.625820933186489
99 : 6.220355825078324
999 : 6.220355825078324
_cdlwmybcp7nsua3dupw2bxgbw

# Bag of Words

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english', lowercase=True)
for i in range(10):
    y = df_data1['Processed_Text'].values[i]
    word_count = cv.fit_transform(y)
    df_ = pd.DataFrame(word_count.toarray(), columns = cv.get_feature_names())
    print(df_)


     000  001  00a  ...  written  yr  zgztrr59odzwhhik021ozlp48ndjjppg1vvozzpz
0      0    0    0  ...        0   0                                         0
1      0    0    0  ...        0   0                                         0
2      0    0    0  ...        0   0                                         0
3      0    0    0  ...        0   0                                         0
4      0    0    0  ...        0   0                                         0
..   ...  ...  ...  ...      ...  ..                                       ...
549    0    0    0  ...        0   0                                         0
550    0    0    0  ...        0   0                                         0
551    0    0    0  ...        0   0                                         0
552    0    0    0  ...        0   0                                         0
553    0    0    0  ...        0   0                                         0

[554 rows x 265 columns]
     00  03  07  0x0  12  

In [11]:
for i in range(10):
  x = df_data1['Processed_Text'].values[i]
  b =nltk.bigrams(x)
  z=[" ".join(t) for t in b]
  df_data1['Processed_Text'].values[i] = z
df_data1.head()

,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...","[2021 coachman, coachman sportscoach, sportsco..."
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...","[jemarl baker, baker Jr, Jr transfer, transfer..."
2,"{""text"": [""Publications | The Protect Heritage...","[public the, the protect, protect heritag, her..."
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...","[art v, v athlet, athlet two, two great, great..."
4,"{\r\n ""title"": [\r\n ""Purposeful Inv...","[purpos invest, invest legaci, legaci bridg, b..."
